# Cuaderno para cargar archivos a MinIO

En este cuaderno se muestran algunas funciones de ayuda y algunos ejemplos para cargar archivos y directorios completos a MinIO.

Este ejemplo asume que ya se ejecutó el cuadreno DescargaDatos.ipynb

## Librerias

In [ ]:
from minio import Minio
from minio.error import S3Error
import os
from minio.deleteobjects import DeleteObject

## Configuración de conexión

In [ ]:
client = Minio(
        "minio:9000",
        access_key="minio",
        secret_key="minio123",
        secure=False
    )

## Funciones de ayuda

Algunas funciones que nos ayudan a subir archivos y directorios completos de manera recursiva

In [ ]:
# Crear una cubeta para almacenar datos
def create_bucket(client, bucket):
    if client.bucket_exists(bucket):
        print("Bucket {} already exists".format(bucket))
    else:
        client.make_bucket(bucket)
    
# Copia un solo archivo 
def copy_file(client,bucket,file_path,dest_folder=""):
    file_name= os.path.basename(file_path)
    client.fput_object(bucket, os.path.join(dest_folder,file_name), file_path)
    #print("File {} copied to {}".format(file_path,os.path.join(dest_folder,file_name)))
    
# Copia todo el contenido de un directorio de manera recursiva manteniendo el esquema de archivos
def copy_folder(client,bucket,folder_path,dest_folder=""):
    folder_name = os.path.basename(folder_path)
    dest_folder = os.path.join(dest_folder, folder_name)
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isdir(file_path): 
            copy_folder(client,bucket,file_path,dest_folder)
        else:
            copy_file(client,bucket,file_path,dest_folder)
            
# Borrra todos los objetos que esten en un directorio de MinIO. USar con mucho cuidado
def delete_objects(client,bucket,prefix=None):
    names = map(
        lambda x: DeleteObject(x.object_name),
        client.list_objects(bucket, prefix, recursive=True)
    )
    for err in client.remove_objects(bucket, names):
        print("Deletion Error: {}".format(err))

## Ejemplo de datos

Aqui se muestra un ejemplo con datos abiertos de INEGI de la encuesta ENIGH. Para descargar y descomprimir los archivos en sus locaciones adecuadas por favor utilice el cuaderno DescargaDatos.ipynb

Para este ejemplo funcional los datos se deben cargar a la capeta warehouse/ de MinIO ya que es la que esta conectada a Hive-Metastore


In [ ]:
copy_folder(client, "hive", "../data/conjunto_de_datos_enigh_2018_ns_csv","warehouse/")
copy_folder(client, "hive", "../data/conjunto_de_datos_enigh2016_nueva_serie_csv","warehouse/")
copy_folder(client, "hive", "../data/enigh_ncv_2014_csv","warehouse/")